# Sentiment analysis using Neural Networks


In this notebook we will perform sentiment analysis using a few simple Neural Network based architectures.
For this problem we use the IMDB Large Movie Review Dataset. The dataset contains 25,000 highly polar movie reviews for both train and test dataset, each with 12,500 positive (greater than equal to 7/10 rating) and 12,500 negative reviews(less than equal to 4/10 rating). 

In [1]:
from os import listdir
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
#from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np

Using TensorFlow backend.


In [4]:
#we retrieve train and test file names

train_dir = "./aclImdb/train/"
test_dir = "./aclImdb/test/"

tr_review = [re_filename for re_filename in listdir(train_dir)]
te_review = [re_filename for re_filename in listdir(test_dir)]

#we initialize the train and test arrays

tr_X = []
tr_Y = []
te_X = []
te_Y = []

#we arrange the reviews into the train and test arrays 

for review_file in tr_review:
    f_review = open(train_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    tr_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        tr_Y.append(1)
    else:
        tr_Y.append(0)
    

for review_file in te_review:
    f_review = open(test_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    te_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        te_Y.append(1)
    else:
        te_Y.append(0)


1
2
3


We will now create the validation set from the train set

Using the last 4 numbers of my uni for the seed value seed to ensure all answers remain unique

In [5]:
#replace 2 (SEED) with the last 4 numbers of your Uni
#Uni: as5196
SEED = 5196
seed_counter = 0
while(1):

    shuffle_combine = list(zip(tr_X, tr_Y))
    random.seed(SEED+seed_counter)
    seed_counter+=1
    random.shuffle(shuffle_combine)

    tr_X, tr_Y = zip(*shuffle_combine)

    val_X = tr_X[:5000]
    val_Y = tr_Y[:5000]

    counter = 0
    for label in val_Y:
        counter+=label

    print (counter)
    print (seed_counter)
    if(counter>2400 and counter <2600):
        tr_X = tr_X[5000:]
        tr_Y = tr_Y[5000:]
        break

2477
1


In [6]:
print("Length of Train review set : " + str(len(tr_X)))
print("Length of Train label set : " + str(len(tr_Y)))
print("Length of Validation review set : " + str(len(val_X)))
print("Length of Validation label set : " + str(len(val_Y)))
print("Length of Test review set : " + str(len(te_X)))
print("Length of Test label set : " + str(len(te_Y)))
print("*****************************************")
print("Some sample Reviews Train sets and their labels")
print(tr_X[0][:150])
print(tr_Y[0])
print(tr_X[1][:150])
print(tr_Y[1])
print(tr_X[2][:150])
print(tr_Y[2])
print(tr_X[3][:150])
print(tr_Y[3])
print(tr_X[4][:150])
print(tr_Y[4])

Length of Train review set : 20000
Length of Train label set : 20000
Length of Validation review set : 5000
Length of Validation label set : 5000
Length of Test review set : 25000
Length of Test label set : 25000
*****************************************
Some sample Reviews Train sets and their labels
This movie is fun to watch. If you liked "Dave" with Kevin Klein, you will get a kick out of this. Think "Dave" gone South American as Dreyfus plays J
1
My Tutor Friend is a well scripted romance comedy movie that has something similar to My Sassy Girl.. there's no kissing/sex scenes. Hollywood should 
1
A meteorite falls in the country of a small town, bringing a jelly creature. An old farmer is attacked by the alien in his hand, and the youths Steve 
1
Very rarely does one come across an indie comedy that leaves a lasting impression. Cross Eyed is a rare gem. The writer director not only tackled the 
1
Well, there's no real plot to speak of, it's just an excuse to show some scenes of ex

In [7]:
#we collect all the reviews from train validation and test set to generate 
texts = []
texts += tr_X 
texts += te_X 
texts += val_X
len(texts)

#we clip the sentence length to first 250 words. 
MAX_SEQUENCE_LENGTH = 250

#length of vocab, Tokenizer will only use vocab_len most common words
vocab_len = 25000

#we tokenize the texts and convert all the words to tokens
tokenizer = Tokenizer(num_words=vocab_len)
tokenizer.fit_on_texts(texts)

token_tr_X = tokenizer.texts_to_sequences(tr_X)
token_te_X = tokenizer.texts_to_sequences(te_X)
token_val_X = tokenizer.texts_to_sequences(val_X)

#to ensure all reviews have the same length, we pad the smaller reviews with 0, 
#and cut the larger reviews to a max length 
#(we clip from the top, as the end of the reviews generally have a conclusion which provides better features)
x_train = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
x_test = sequence.pad_sequences(token_te_X, maxlen=MAX_SEQUENCE_LENGTH)
x_val = sequence.pad_sequences(token_val_X, maxlen=MAX_SEQUENCE_LENGTH)


#changes the labels to one-hot encoding
y_train = np_utils.to_categorical(tr_Y)
y_test = np_utils.to_categorical(te_Y)
y_val = np_utils.to_categorical(val_Y)


In [8]:
print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)
print('X_val shape:', x_val.shape)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print('y_val shape:', y_val.shape)


print("*****************************************")
print("Tokenized Reviews Train sets and their labels")
print(x_train[0][:20])
print(y_train[0])
print()
print(x_train[1][:20])
print(y_train[1])
print()
print(x_train[2][:20])
print(y_train[2])
print()
print(x_train[3][:20])
print(y_train[3])
print()
print(x_train[4][:20])
print(y_train[4])
print()

X_train shape: (20000, 250)
X_test shape: (25000, 250)
X_val shape: (5000, 250)
y_train shape: (20000, 2)
y_test shape: (25000, 2)
y_val shape: (5000, 2)
*****************************************
Tokenized Reviews Train sets and their labels
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]



********************************************

As you can see the reviews have now been transformed into indices to tokenized vocabulary and the labels have been converted to one-hot encoding. We can now go ahead and feed these sequences to Neural Network Models.

********************************************

# Part A

Building your first model (5 Points)

Construct this sequential model using Keras :

![title](img/model1.jpg)

In [7]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(25000, 128, input_length = 250))
model.add(Flatten())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## compile it here according to instructions
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               6400200   
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 9,600,602
Trainable params: 9,600,602
Non-trainable params: 0
___________________________________________________

In [8]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 152s 8ms/step - loss: 0.4049 - acc: 0.7998 - val_loss: 0.2949 - val_acc: 0.8724
Epoch 2/4
20000/20000 [==============================] - 154s 8ms/step - loss: 0.0448 - acc: 0.9863 - val_loss: 0.4475 - val_acc: 0.8550
Epoch 3/4
20000/20000 [==============================] - 143s 7ms/step - loss: 0.0030 - acc: 0.9992 - val_loss: 0.5539 - val_acc: 0.8644
Epoch 4/4
20000/20000 [==============================] - 149s 7ms/step - loss: 3.2458e-04 - acc: 1.0000 - val_loss: 0.6099 - val_acc: 0.8682


# Part B

Stacking Fully Connected Layers (5 points)

Construct this sequential model using Keras :

![title](img/model2.jpg)

In [9]:
print('Build model...')

## implement model here

model = Sequential()

model.add(Embedding(25000, 128, input_length = 250))
model.add(Flatten())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## compile it here according to instructions
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               6400200   
_________________________________________________________________
activation_3 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
activation_4 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 402   

In [10]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 158s 8ms/step - loss: 0.4020 - acc: 0.8037 - val_loss: 0.2981 - val_acc: 0.8768
Epoch 2/4
20000/20000 [==============================] - 163s 8ms/step - loss: 0.0446 - acc: 0.9849 - val_loss: 0.4134 - val_acc: 0.8490
Epoch 3/4
20000/20000 [==============================] - 162s 8ms/step - loss: 0.0067 - acc: 0.9979 - val_loss: 0.6646 - val_acc: 0.8552
Epoch 4/4
20000/20000 [==============================] - 153s 8ms/step - loss: 0.0063 - acc: 0.9979 - val_loss: 0.7838 - val_acc: 0.8422


# Part C

Using LSTMS based networks(5 Points) 

Construct this sequential model using Keras :

![title](img/model3.jpg)

In [11]:
print('Build model...')

## implement model here

model = Sequential()

model.add(Embedding(25000, 128, input_length = 250))
#model.add(Flatten())
model.add(LSTM(128))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## compile it here according to instructions
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_6 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_7 (Activation)    (None, 2)                 0         
Total params: 3,348,354
Trainable params: 3,348,354
Non-trainable params: 0
___________________________________________________

In [12]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 490s 25ms/step - loss: 0.4418 - acc: 0.7871 - val_loss: 0.3692 - val_acc: 0.8620
Epoch 2/5
20000/20000 [==============================] - 486s 24ms/step - loss: 0.2176 - acc: 0.9170 - val_loss: 0.3466 - val_acc: 0.8588
Epoch 3/5
20000/20000 [==============================] - 482s 24ms/step - loss: 0.1158 - acc: 0.9603 - val_loss: 0.3748 - val_acc: 0.8670
Epoch 4/5
20000/20000 [==============================] - 491s 25ms/step - loss: 0.0778 - acc: 0.9738 - val_loss: 0.5051 - val_acc: 0.8254
Epoch 5/5
20000/20000 [==============================] - 487s 24ms/step - loss: 0.0843 - acc: 0.9698 - val_loss: 0.6400 - val_acc: 0.8492


# Part D

Adding Pretrained Word Embeddings(10 Points)

Construct this sequential model using Keras :

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model4.jpg)

In [10]:
import codecs

#dimension of Glove Embeddings.
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

#load glove embeddings
gembeddings_index = {}
with codecs.open('glove.42B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

# nb_words contains the total length of vocab
nb_words = len(word_index) +1

#get glove embeddings for each word in tokenizer.
#g_word_embedding_matrix holds the embeddings dictionary
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
#total words in the tokenizer not in Embedding matrix
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))



Found 124252 unique tokens
G Word embeddings: 1176088
G Null word embeddings: 38684


In [10]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(nb_words, 300, weights = [g_word_embedding_matrix]))
model.add(LSTM(128, recurrent_dropout = 0.2))
model.add(Dropout(0.2))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

## to use the glove embeddings, your embedding layer would take the vocab size as input dimension, 
## Glove embedding dimension as the output dimsion
## and you will provide the  embedding dictionary as the 'weights' parameter (!important) to the embedding layer.


## compille it here according to instructions
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         37275900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 37,512,318
Trainable params: 37,512,318
Non-trainable params: 0
_________________________________________________________________
Model Built


In [11]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 412s 21ms/step - loss: 0.4106 - acc: 0.8123 - val_loss: 0.3109 - val_acc: 0.8696
Epoch 2/5
20000/20000 [==============================] - 412s 21ms/step - loss: 0.2014 - acc: 0.9229 - val_loss: 0.2538 - val_acc: 0.8978
Epoch 3/5
20000/20000 [==============================] - 411s 21ms/step - loss: 0.0882 - acc: 0.9716 - val_loss: 0.3356 - val_acc: 0.8860
Epoch 4/5
20000/20000 [==============================] - 411s 21ms/step - loss: 0.0342 - acc: 0.9906 - val_loss: 0.3951 - val_acc: 0.8876
Epoch 5/5
20000/20000 [==============================] - 411s 21ms/step - loss: 0.0180 - acc: 0.9945 - val_loss: 0.5655 - val_acc: 0.8888


# Dont attempt this

Stacking LSTM layers

Unfortunately it takes very long to train, be aware we can stack LTMSs over each other like this.
This requires bottom LSTM to return a sequences instead instead of single vector, which becomes input for the top LSTM.


![title](img/model5.jpg)

In [12]:
from keras.layers import Conv1D

# Part E

Using Convolutional Networks (10 points)

Construct the model, shown below. Use the same loss functions and optimizers as before

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model6.jpg)

In [13]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(nb_words, 300, weights = [g_word_embedding_matrix], input_length = 250))
model.add(Conv1D(filters = 128, kernel_size = 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Conv1D(filters = 64, kernel_size = 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Conv1D(filters = 32, kernel_size = 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## compille it here according to instructions
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
activation_1 (Activation)    (None, 248, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_2 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
activation_2 (Activation)    (None, 246, 64)           0     

In [17]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 244s 12ms/step - loss: 0.4461 - acc: 0.7661 - val_loss: 0.2576 - val_acc: 0.8978
Epoch 2/5
20000/20000 [==============================] - 244s 12ms/step - loss: 0.2204 - acc: 0.9153 - val_loss: 0.2505 - val_acc: 0.8978
Epoch 3/5
20000/20000 [==============================] - 245s 12ms/step - loss: 0.1235 - acc: 0.9532 - val_loss: 0.2973 - val_acc: 0.8912
Epoch 4/5
20000/20000 [==============================] - 243s 12ms/step - loss: 0.0768 - acc: 0.9730 - val_loss: 0.3269 - val_acc: 0.8920
Epoch 5/5
20000/20000 [==============================] - 244s 12ms/step - loss: 0.0446 - acc: 0.9838 - val_loss: 0.6186 - val_acc: 0.8720


# Part F

Model constructed : (5 points)

Test Accuracy Over 87.5%: (5 Points)

Bonus: Min(10, Square of (test_score - 88%))

Create your best model, use Validation score to judge your best model and check accuracy on test set


### BEST MODEL

In [14]:
print('Build model...')

## implement model here - experimenting with conv

model = Sequential()
model.add(Embedding(nb_words, 300, weights = [g_word_embedding_matrix], input_length = 250))
model.add(Conv1D(filters = 128, kernel_size = 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Conv1D(filters = 64, kernel_size = 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Conv1D(filters = 32, kernel_size = 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## to use the glove embeddings, your embedding layer would take the vocab size as input dimension, 
## Glove embedding dimension as the output dimsion
## and you will provide the  embedding dictionary as the 'weights' parameter (!important) to the embedding layer.

## compile it here according to instructions
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
activation_6 (Activation)    (None, 248, 128)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_6 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
activation_7 (Activation)    (None, 246, 64)           0     

You can keep saving models with different names in model_name, 

so you can retrieve their weights again for testing, you dont have to retrain 
(You would have to initialize the model definition again).

In [15]:
wt_dir = "./weights/"
model_name = 'model_best'
early_stopping =EarlyStopping(monitor='val_acc', patience=2)
bst_model_path = wt_dir + model_name + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, monitor='val_acc', save_best_only=True, save_weights_only=True)

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=7,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True,
         callbacks=[early_stopping, model_checkpoint])


Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/7
20000/20000 [==============================] - 247s 12ms/step - loss: 0.4384 - acc: 0.7780 - val_loss: 0.3090 - val_acc: 0.8702
Epoch 2/7
20000/20000 [==============================] - 246s 12ms/step - loss: 0.2313 - acc: 0.9094 - val_loss: 0.2426 - val_acc: 0.9040
Epoch 3/7
20000/20000 [==============================] - 244s 12ms/step - loss: 0.1383 - acc: 0.9485 - val_loss: 0.2739 - val_acc: 0.9036
Epoch 4/7
20000/20000 [==============================] - 245s 12ms/step - loss: 0.0857 - acc: 0.9689 - val_loss: 0.2900 - val_acc: 0.9022


If you plan on using Ensemble averaging, feel free to edit the code below or add multiple models.

Make sure they get saved and can be retrieved when executing serially.

In [17]:
model.load_weights(bst_model_path)
scores = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 34s 1ms/step
Accuracy: 89.57%


Simple Conv1D model - stored as model1.h5 in weights directory - 89.57%

# Part G

Explain how Dense, LSTM and Convolution Layers work.

Explain Relu, Dropout, and Softmax work.

Analyze the architectures you constructed, with the accuracies you achieved and the training time it took. 

What are some insights you gained with these experiments? 

(5 Points)


### Layers

##### 1.Dense Layers: 
A dense layer is a layer of neurons which is fully connected. A linear operation in which every input is connected to every output by a weight (so there are n_inputs * n_outputs weights); followed by a non-linear activation function. The values in the weight matrix are the trainable parameters which get updated during backpropagation. A dense layer thus is used to change the dimensions of the input vector. Mathematically, it applies a rotation, scaling, translation transform to the vector.

In Keras, Dense implements the operation: output = activation(dot(input, kernel) + bias) where activation is the element-wise activation function passed as the activation argument, kernel is a weights matrix created by the layer, and bias is a bias vector created by the layer (if use_bias is True).
 
 
##### 2.LSTM Layers: 
Long short-term memory (LSTM) block or network is a simple recurrent neural network which can be used as a building component or block (of hidden layers) for an eventually bigger recurrent neural network.The cell is responsible for "remembering" values over arbitrary time intervals. An LSTM block is composed of four main components: a cell, an input gate, an output gate and a forget gate. The cell is responsible for "remembering" values over arbitrary time intervals. Each of the three gates can be thought as a "conventional" artificial neuron, as in a multi-layer (or feedforward) neural network: that is, they compute an activation of a weighted sum. There are connections between these gates and the cell. Some of the connections are recurrent, some of them are not. An LSTM is well-suited to classify, process and predict time series given time lags of unknown size and duration between important events.


##### 3.Convolution Layers: 
A linear operation using a subset of the weights of a dense layer. Nearby inputs are connected to nearby outputs. The weights for the convolutions at each location are shared. Due to the weight sharing, and the use of a subset of the weights of a dense layer, there's far less weights than in a dense layer. They are followed by a non-linear activation function. This layer creates a convolution kernel that is convolved with the layer input over a single spatial (or temporal) dimension to produce a tensor of outputs (1-D convolutionl layer). If use_bias is True, a bias vector is created and added to the outputs. Finally, if activation is not None, it is applied to the outputs as well. 


##### 4. Relu (Rectified Linear unit): 
Relu is a commonly used activation functions in neural nets. The ReLU function is f(x)=max(0,x). Usually this is applied element-wise to the output of some other function, such as a matrix-vector product. In MLP usages, rectifier units replace all other activation functions except perhaps the output layer. 
One way ReLUs improve neural networks is by speeding up training. The gradient computation is very simple (either 0 or 1 depending on the sign of xx). Also, the computational step of a ReLU is easy: any negative elements are set to 0.0. Logistic and hyperbolic tangent networks suffer from the vanishing gradient problem, where the gradient essentially becomes 0 after a certain amount of training (because of the two horizontal asymptotes) and stops all learning in that section of the network. ReLU units are only 0 gradient on one side, which empirically is superior.


##### 5. Softmax:
The softmax function is commonly used in the final layer of a neural network-based classifier. Such networks are commonly trained under a log loss (or cross-entropy) regime, giving a non-linear variant of multinomial logistic regression. softmax function can be used to represent a categorical distribution – that is, a probability distribution over K different possible outcomes.The function itself is a generalization of the logistic function that "squashes" a K-dimensional vector of arbitrary real values to a K-dimensional vector of real values in the range [0, 1] that add up to 1. 


##### 6. Dropout: 
Dropout is a technique where randomly selected neurons are ignored during training. They are “dropped-out” randomly. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass.Weights of neurons are tuned for specific features providing some specialization. Neighboring neurons become to rely on this specialization, which if taken too far can result in a fragile model too specialized to the training data. If neurons are randomly dropped out of the network during training, that other neurons will have to step in and handle the representation required to make predictions for the missing neurons. The effect is that the network becomes less sensitive to the specific weights of neurons. This in turn results in a network that is capable of better generalization and is less likely to overfit the training data.





### Analysis of architectures & Insights
Part A creates a very simple model with 3 layers. The embedding layer serves to create an input layer by creating embeddings of size 128 which is then flattened to create a layer of 32000 neurons to represent the input. There is dense hidden layer of 200 neurons with a relu activation. The output layer is a dense layer with 2 output neurons and softmax activation to determine the probability of being a positive or negative review. There are 9,600,602 trainable parameters.Each of the 4 epochs take 150secs and validation accuracy increases with increasing epochs. Max val accuracy though is after 1st epoch at 0.87.


PartB creates a similar network with an additional hidden dense layer of 200 neurons with relu activation. There epochs seem to take slightly longer to train, though overall validation accuracy does not seem to increase as compared to PartA. After 4 epochs val accuracy is actually lower than PartA and the model seem to be overfitting.No. of trainable parameters is 9,640,802. The additional 40,000 neurons take more time to train, add complexity but do not improve accuracy.


PartC uses an LSTM layer of size 128 and a dense layer of size 128 as hidden layers.No. of trainable parameters is 3,348,354 which is much lesser than both part A and B. However this model takes much longer to train (~485secs per epoch). Val accuracy after first epoch is similar to PartA and PartB. After 5 epochs its validation accuracy is still lower than the simplest net in Part A.


PartD is a more complex model and uses 300 dimensional Glove word embeddings for input. It uses an LSTM layer with recurrent dropout and an additional Dropout layer in comparison to partC. No. of trainable parameters increase massively to 37M. However each epoch takes approx. 411 secs to train which is faster than PartC (because of recurrent dropout and dropout). Val accuracy increases as compared to previous models and is maximum after 2nd epoch at 0.897.


PartE uses a much more complex model with ~39M trainabale parameters.It uses 3 convolutional 1D layers of decreasing filter size and 4 dropout layers of 20%. Hidden layers use relu activation and output uses softmax as with the other cases.On average each epoch takes 244secs to train, which is much lower than part C and D despite more trainable parameters. Model has highest accuracy after first 2 epochs of 0.897 after which val accuracy starts going down (due to overfitting to training set).

It seems in most cases, the validation accuracy seems to go down after first 1-2 epochs due to overfitting to training set. Also simply adding more dense layers merely increases training time but does not give significant increase in accuracy. LSTM models take a very long time to train. In our case, Part E with dropout and convolutional layers gave best performance. 1D convolutional layers were thus used in the best model similar to PartE. Adding more dense layers and increasing dropout did not improve performance. Best accuracy obtained on test set was 89.57%. 
